# Name: Margaret Nguyen

# Machine Learning: Ordinary least squares (OLS)

**Assignment: Create an Ordinary Least Squares model with cyclist deaths and injuries per capita as the target variable. Use per capita parameters in the model and employ data from both Massachusetts and Pennsylvania.**

In [ ]:
# Import packages
import numpy as np # v 1.21.5
import sklearn # v 1.0.2
import pandas as pd # v 1.4.4
import ydata_profiling as pp # v 3.6.6
import statsmodels.api as sm # v 0.13.2

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_percentage_error

# Ploting libraries 
import matplotlib.pyplot as plt # v 3.5.2
import seaborn as sns # v 0.11.2

## I. df_pa_crash.csv (Pennsylvannia)

### 1. Load and clean data

In [ ]:
# Read the csv file 
df_pa_crash = pd.read_csv('/Users/margaret06/Documents/GitHub/Carlisle_Borough_Transportation_Study/data/df_pa_crash.csv')

# Clean datasets
df_pa_crash = df_pa_crash.drop(columns = ['Unnamed: 0'])

# Select columns with numeric data types (int or float) using select_dtypes
numeric_columns = df_pa_crash.select_dtypes(include=['number'])

# Create a new DataFrame with only the numeric columns
df_pa_filtered = df_pa_crash[numeric_columns.columns]

# Drop unnessary columns
df_pa_filtered = df_pa_filtered.drop(['PENN_DOT_MUNI_ID', 'state', 'county', 'county_subdivision', 'LAND_AREA.1', 'PENN_DOT_COUNTY_NUM', 'FEDERAL_EIN_CODE', 'HOME_RULE_YEAR', 'INCORPORATION_YEAR', 'MUNICIPALITY'], axis=1)

# Replace NaN values with 0 in the entire DataFrame
df_pa_filtered = df_pa_filtered.fillna(0)

# Rename BNA Score column to BNA_SCORE column
df_pa_filtered.rename(columns={'BNA Score': 'BNA_SCORE'}, inplace=True)

# Reset index
df_pa_filtered.reset_index(inplace = True, drop = True)

In [ ]:
# Define the columns for which you want to calculate per capita values
columns_to_convert = [
    'LAND_AREA', 'BIKE_TO_WORK_EST', 'BIKE_TO_WORK_MARG',
    'WALK_TO_WORK_EST', 'WALK_TO_WORK_MARG', 'DRIVE_SOLO_TO_WORK_EST',
    'DRIVE_SOLO_TO_WORK_MARG', 'CARPOOL_TO_WORK_EST',
    'CARPOOL_TO_WORK_MARG', 'PUBTRANS_TO_WORK_EST',
    'PUBTRANS_TO_WORK_MARG', 'EMPLOYEES_FULL_TIME',
    'EMPLOYEES_PART_TIME', 'AUTOMOBILE_COUNT',
    'BICYCLE_BY_AUTO_COUNT', 'BICYCLE_DEATH_BY_AUTO_COUNT',
    'BICYCLE_SUSP_SERIOUS_INJ_BY_AUTO_COUNT', 'PED_BY_AUTO_COUNT',
    'PED_DEATH_BY_AUTO_COUNT', 'PED_SUSP_SERIOUS_INJ_BY_AUTO_COUNT',
    'BICYCLE_SOLO_COUNT', 'BICYCLE_DEATH_SOLO_COUNT',
    'BICYCLE_SUSP_SERIOUS_INJ_SOLO_COUNT', 'PED_SOLO_COUNT',
    'PED_DEATH_SOLO_COUNT', 'PED_SUSP_SERIOUS_INJ_SOLO_COUNT'
]

# Create new columns with "_PER_CAPITA" suffix by dividing each column by 'POPULATION'
for column in columns_to_convert:
    new_column_name = column + '_PER_CAPITA'
    df_pa_filtered[new_column_name] = df_pa_filtered[column] / df_pa_filtered['POPULATION']

**Perform Exploratory Data Analysis (EDA) to check for multicollinearity among the independent variables in the dataset**

In [ ]:
# df_pa_filtered.columns
df_pa_filtered.shape, df_pa_filtered.dtypes

In [ ]:
df_pa_filtered.describe()

In [ ]:
# Check for NaN missing values
df_pa_filtered.isna().sum()

In [ ]:
df_pa_filtered.head(3)

**Use Pandas Profiling**

In [ ]:
pp.ProfileReport(df_pa_filtered)

**According to the Pandas Profiling report, we can see that our independent variables are highly correlated with each other. This suggests a high likelihood of multicollinearity in our linear regression model. To address this issue, I need to remove some independent variables and retain only the necessary ones as much as possible. Additionally, I have decided to remove BICYCLE_DEATH_BY_AUTO_COUNT and BICYCLE_DEATH_SOLO_COUNT because both exhibit a high level of imbalance (62.6%).**

In [ ]:
# Drop independent variables that are highly correlated
df_pa_filtered = df_pa_filtered.drop(['LAND_AREA', 'BIKE_TO_WORK_EST', 'BIKE_TO_WORK_MARG',
        'WALK_TO_WORK_EST', 'WALK_TO_WORK_MARG', 'DRIVE_SOLO_TO_WORK_EST',
        'DRIVE_SOLO_TO_WORK_MARG', 'CARPOOL_TO_WORK_EST',
        'CARPOOL_TO_WORK_MARG', 'PUBTRANS_TO_WORK_EST', 'PUBTRANS_TO_WORK_MARG',
        'EMPLOYEES_FULL_TIME', 'EMPLOYEES_PART_TIME', 'AUTOMOBILE_COUNT',
        'BICYCLE_BY_AUTO_COUNT', 'BICYCLE_DEATH_BY_AUTO_COUNT',
        'BICYCLE_SUSP_SERIOUS_INJ_BY_AUTO_COUNT', 'PED_BY_AUTO_COUNT',
        'PED_DEATH_BY_AUTO_COUNT', 'PED_SUSP_SERIOUS_INJ_BY_AUTO_COUNT',
        'BICYCLE_SOLO_COUNT', 'BICYCLE_DEATH_SOLO_COUNT',
        'BICYCLE_SUSP_SERIOUS_INJ_SOLO_COUNT', 'PED_SOLO_COUNT',
        'PED_DEATH_SOLO_COUNT', 'PED_SUSP_SERIOUS_INJ_SOLO_COUNT'], axis=1)

### 2. Fit the linear regression using [scikit-learn](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html)

**Separate data set in Y(independent) and X (dependent) variable**

In [ ]:
y = df_pa_filtered["BICYCLE_BY_AUTO_COUNT_PER_CAPITA"] # Y = df_pa_filtered.BICYCLE_BY_AUTO_COUNT_PER_CAPITA
X = df_pa_filtered.loc[:, df_pa_filtered.columns != "BICYCLE_BY_AUTO_COUNT_PER_CAPITA"] # I want all columns except the BICYCLE_BY_AUTO_COUNT_PER_CAPITA column which is the dependent variable

**Use the train_test_split function to split your data into training (80%) and testing set (20%)**

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=5)

**Fit, run or estimate the regression model**

In [ ]:
model = LinearRegression() # Create an instance of the linear regression class
model.fit(X_train, y_train)

In [ ]:
# Coefficient values
model.coef_

In [ ]:
print(model.intercept_, model.coef_,model.score(X_test, y_test)) # Model score gives you the R-squared

**Using the independent variables in the testing set, to predict the dependent variables**

In [ ]:
y_pred = model.predict(X_test)

**Use the test set, check how well my model does in terms of error metrics**

In [ ]:
MAE = mean_absolute_error(y_test,y_pred)
MSE = mean_squared_error(y_test,y_pred)
MAPE = mean_absolute_percentage_error(y_test,y_pred)
MSE, MAE, MAPE

### 3. Fit the linear regression using [Statsmodels](https://www.statsmodels.org/stable/index.html)

In [ ]:
X_train = sm.add_constant(X_train)

In [ ]:
model2 = sm.OLS(y_train, X_train).fit()

In [ ]:
print(model2.summary())

**In-sample prediction**

In [ ]:
ypred2 = model2.predict(X_train)
model2.params

**According to the results from Statsmodels:**
- **The estimated coefficients, which include DRIVE_SOLO_TO_WORK_EST_PER_CAPITA, CARPOOL_TO_WORK_MARG_PER_CAPITA, AUTOMOBILE_COUNT_PER_CAPITA, PED_BY_AUTO_COUNT_PER_CAPITA, BICYCLE_DEATH_SOLO_COUNT_PER_CAPITA, and PED_SUSP_SERIOUS_INJ_SOLO_COUNT_PER_CAPITA, are statistically significant at the 5% level of significance in the 90% training set.**
- **Additionally, in the 80% training set, the estimated coefficients, including DRIVE_SOLO_TO_WORK_EST_PER_CAPITA, AUTOMOBILE_COUNT_PER_CAPITA, and PED_BY_AUTO_COUNT_PER_CAPITA, are statistically significant at the 5% level of significance.**
- **However, in the 70% training set, none of the estimated coefficients are statistically insignificant.**

### 4. Plot the residuals, display feature coefficients, and create a QQ plot

### Credit:

The following code is based on the work of my supervisor, Mitch Shiles. The original code can be found at this link: [Mitch Shiles' GitHub](https://github.com/rmshiles/Custom-Data-Tools/blob/main/testing%20for%20normality%20.ipynb).

**Below is a defined function designed to test residuals for a normal distribution.**

In [ ]:
# Define a test for normality 

# Test for normality 
# y_test is the target variable and y_pred are the predicted variables 
def test_for_normality(y_test,y_pred):
    from scipy.stats import boxcox
    from scipy.stats import jarque_bera
    from scipy.stats import normaltest
    colo = np.random.randint(3, size=1)
    colors=[['r','gold','c','m'],
            ['g','orange','b','hotpink'],
            ['skyblue','coral','lightgreen','mediumslateblue'],
           ['g','limegreen','orange','yellow']]
    
    try:
        data_series = y_pred-y_test
    except:
        data_series=y_test
    # Input the mean, standard deviation and lenght of the residuals
    normal = np.random.normal(np.mean(data_series), np.std(data_series), len(data_series))

    plt.figure(figsize=(16, 12))

    plt.subplot(2, 2, 1)
    plt.hist(data_series,color = colors[colo[0]][1],alpha = 0.8) #bins=40,
    plt.hist(normal,color = colors[colo[0]][2], alpha = 0.2)

    # Generate a Box Plot of solar system counts
    plt.subplot(2, 2, 2)
    plt.boxplot(data_series)

    # Generate a QQ plot of the gamma distribution and the solar system counts 
    plt.subplot(2, 1, 2)
    orderd_normal = sorted(normal)
    ordered_data=sorted(data_series)
    plt.scatter(ordered_data,orderd_normal, color = colors[colo[0]][3])
    plt.plot(orderd_normal,orderd_normal,color= colors[colo[0]][0])
    plt.title('QQPlot of residuals and normal Distribution')
    plt.xlabel('residuals')
    plt.ylabel('normaly distribution')
    plt.show()

    jb_stats = jarque_bera(data_series)
    norm_stats = normaltest(data_series)
    print('the Jarque berra stat is {}, and the pvalue is {}'.format(jb_stats[0],jb_stats[1]))
    print(norm_stats)
    
    # elecResiduals = np.sort(result2_elect.resid[np.logical_not(np.isnan(result2_elect.resid))])
    sorted_data_series = np.sort(data_series[np.logical_not(np.isnan(data_series))])

    data_series_min = sorted_data_series.min()
    data_series_max = sorted_data_series.max()
    data_series_len = len(sorted_data_series)
    data_series_std = np.std(sorted_data_series)
    data_series_avg = np.mean(sorted_data_series)

    print('the Minimum is {}'.format(data_series_min))
    print('the Maximum is {}'.format(data_series_max))
    print('the Length is {}'.format(data_series_len))
    print('the Length is {}'.format(data_series_len))
    print('the Standard Deviation is {}'.format(data_series_std))
    print('the Mean is {}'.format(data_series_avg))
    print('\n')

In [ ]:
import matplotlib
matplotlib.use('Qt5Agg')  # Use an appropriate backend like 'Qt5Agg' for GUI display
import matplotlib.pyplot as plt

In [ ]:
# Plot the residuals, display feature coefficients, and create a QQ plot
test_for_normality(y_test,y_pred)

**The warning regarding the kurtosis test indicates that it's typically valid for larger sample sizes (n >= 20). With a sample size of 9, it's recommended to use other statistical tests or methods to assess kurtosis accurately.**

## II. df_mass_bna.csv (Massachusetts)

### 1. Load and clean data

In [ ]:
# Create a blank dataframe
df_mass_bna = pd.DataFrame()

# Read the csv file 
df_mass_bna = pd.read_csv('/Users/margaret06/Documents/GitHub/Carlisle_Borough_Transportation_Study/data/df_mass_bna.csv')

# Clean datasets
df_mass_crash = df_mass_bna.drop(columns = ['Unnamed: 0'])

# Exclude the NaN from 'VEHC_CONFIG_CL'
df_mass_crash = df_mass_crash[df_mass_crash['VEHC_CONFIG_CL'].notna()]

# List of NOT automobiles: Snowmobile, Moped, Motorcycle, Other Light Trucks (10,000 lbs., or Less), Other e.g. Farm Equipment, Unknown.
# Exclude the non-automobiles from 'VEHC_CONFIG_CL' columns
list_non_automobiles = ['V1:(Unknown vehicle configuration)', 'V1:(Other e.g. farm equipment)', 'V1:(Unknown vehicle configuration) / V2:(Unknown vehicle configuration)']
df_mass_crash= df_mass_crash[~df_mass_crash['VEHC_CONFIG_CL'].isin(list_non_automobiles)]

In [ ]:
# Fatal - injuries that resulted in death 
# Incapacitating - serious injuries require immediate medical attention

## BICYCLE_DEATH_BY_AUTO_COUNT
# Filter the DataFrame for cyclist fatalities
cyclist_fatalities = df_mass_crash[(df_mass_crash['INJY_STAT_DESCR'] == 'Fatal injury (K)') & (df_mass_crash['NON_MTRST_TYPE_CL'] == 'Cyclist')]

# Group the filtered DataFrame by 'CITY_TOWN_NAME' and calculate the count for each city
bicycle_death_counts = cyclist_fatalities.groupby('CITY_TOWN_NAME').size().reset_index(name='BICYCLE_DEATH_BY_AUTO_COUNT')

# Merge the bicycle_death_counts DataFrame into df_mass_crash on 'CITY_TOWN_NAME'
df_mass_crash = df_mass_crash.merge(bicycle_death_counts, on='CITY_TOWN_NAME', how='left')

## BICYCLE_SUSP_SERIOUS_INJ_BY_AUTO_COUNT
cyclist_incapacitating = df_mass_crash[(df_mass_crash['INJY_STAT_DESCR'] == 'Non-fatal injury - Incapacitating') & (df_mass_crash['NON_MTRST_TYPE_CL'] == 'Cyclist')]
bicycle_sus_serious_inj_counts = cyclist_incapacitating.groupby('CITY_TOWN_NAME').size().reset_index(name='BICYCLE_SUSP_SERIOUS_INJ_BY_AUTO_COUNT')

# Merge the bicycle_sus_serious_inj_counts DataFrame into df_mass_crash on 'CITY_TOWN_NAME'
df_mass_crash = df_mass_crash.merge(bicycle_sus_serious_inj_counts, on='CITY_TOWN_NAME', how='left')

# Replace NaN values with 0 in the specified columns
df_mass_crash['BICYCLE_DEATH_BY_AUTO_COUNT'].fillna(0, inplace=True)
df_mass_crash['BICYCLE_SUSP_SERIOUS_INJ_BY_AUTO_COUNT'].fillna(0, inplace=True)

## BICYCLE_BY_AUTO_COUNT
df_mass_crash['BICYCLE_BY_AUTO_COUNT'] = df_mass_crash['BICYCLE_DEATH_BY_AUTO_COUNT'] + df_mass_crash['BICYCLE_SUSP_SERIOUS_INJ_BY_AUTO_COUNT']

## AUTOMOBILE_COUNT
auto_count = df_mass_crash.groupby('CITY_TOWN_NAME')['NUMB_VEHC'].sum().reset_index()
auto_count.rename(columns={'NUMB_VEHC': 'AUTOMOBILE_COUNT'}, inplace=True)
# Merge the auto_count into df_mass_crash on 'CITY_TOWN_NAME'
df_mass_crash = df_mass_crash.merge(auto_count, on='CITY_TOWN_NAME', how='left')

## PED_DEATH_BY_AUTO_COUNT
# Filter the DataFrame for pedestrian fatalities
ped_fatalities = df_mass_crash[(df_mass_crash['INJY_STAT_DESCR'] == 'Fatal injury (K)') & (df_mass_crash['NON_MTRST_TYPE_CL'] == 'Pedestrian')]

# Group the filtered DataFrame by 'CITY_TOWN_NAME' and calculate the count for each city
ped_death_counts = cyclist_fatalities.groupby('CITY_TOWN_NAME').size().reset_index(name='PED_DEATH_BY_AUTO_COUNT')

# Merge the ped_death_counts DataFrame into df_mass_crash on 'CITY_TOWN_NAME'
df_mass_crash = df_mass_crash.merge(ped_death_counts, on='CITY_TOWN_NAME', how='left')

## PED_SUSP_SERIOUS_INJ_BY_AUTO_COUNT
ped_incapacitating = df_mass_crash[(df_mass_crash['INJY_STAT_DESCR'] == 'Non-fatal injury - Incapacitating') & (df_mass_crash['NON_MTRST_TYPE_CL'] == 'Pedestrian')]
ped_sus_serious_inj_counts = ped_incapacitating.groupby('CITY_TOWN_NAME').size().reset_index(name='PED_SUSP_SERIOUS_INJ_BY_AUTO_COUNT')

# Merge the ped_sus_serious_inj_counts DataFrame into df_mass_crash on 'CITY_TOWN_NAME'
df_mass_crash = df_mass_crash.merge(ped_sus_serious_inj_counts, on='CITY_TOWN_NAME', how='left')

# Replace NaN values with 0 in the specified columns
df_mass_crash['PED_DEATH_BY_AUTO_COUNT'].fillna(0, inplace=True)
df_mass_crash['PED_SUSP_SERIOUS_INJ_BY_AUTO_COUNT'].fillna(0, inplace=True)

##PED_BY_AUTO_COUNT
df_mass_crash['PED_BY_AUTO_COUNT'] = df_mass_crash['PED_DEATH_BY_AUTO_COUNT'] + df_mass_crash['PED_SUSP_SERIOUS_INJ_BY_AUTO_COUNT']

# Drop the duplicated rows
df_mass_crash = df_mass_crash.drop_duplicates(subset=['CITY_TOWN_NAME', 'BNA Score', 'POPULATION', 
                                                      'BIKE_TO_WORK_EST', 'BICYCLE_BY_AUTO_COUNT', 
                                                      'BICYCLE_DEATH_BY_AUTO_COUNT', "AUTOMOBILE_COUNT"
                                                      'BICYCLE_SUSP_SERIOUS_INJ_BY_AUTO_COUNT', 
                                                      'AUTOMOBILE_COUNT', 'PED_BY_AUTO_COUNT', 
                                                      'PED_DEATH_BY_AUTO_COUNT', 'PED_SUSP_SERIOUS_INJ_BY_AUTO_COUNT'])

In [ ]:
# Fix error
df_mass_crash = df_mass_crash.copy()

# Select columns with numeric data types (int or float) using select_dtypes
numeric_columns = df_mass_crash.select_dtypes(include=['number'])

# Create a new DataFrame with only the numeric columns
df_mass_crash = df_mass_crash[numeric_columns.columns]

# Rename BNA Score column to BNA_SCORE column
df_mass_crash.rename(columns={'BNA Score': 'BNA_SCORE'}, inplace=True)

# Reset index
df_mass_crash.reset_index(drop = True, inplace = True)

In [ ]:
# Define the columns for which you want to calculate per capita values
columns_to_convert = [
    'BIKE_TO_WORK_EST', 'BIKE_TO_WORK_MARG',
    'WALK_TO_WORK_EST', 'WALK_TO_WORK_MARG', 'DRIVE_SOLO_TO_WORK_EST',
    'DRIVE_SOLO_TO_WORK_MARG', 'CARPOOL_TO_WORK_EST',
    'CARPOOL_TO_WORK_MARG', 'PUBTRANS_TO_WORK_EST',
    'PUBTRANS_TO_WORK_MARG', 'AUTOMOBILE_COUNT',
    'BICYCLE_BY_AUTO_COUNT', 'BICYCLE_DEATH_BY_AUTO_COUNT',
    'BICYCLE_SUSP_SERIOUS_INJ_BY_AUTO_COUNT', 'PED_BY_AUTO_COUNT',
    'PED_DEATH_BY_AUTO_COUNT', 'PED_SUSP_SERIOUS_INJ_BY_AUTO_COUNT']

# Create new columns with "_PER_CAPITA" suffix by dividing each column by 'POPULATION'
for column in columns_to_convert:
    new_column_name = column + '_PER_CAPITA'
    df_mass_crash[new_column_name] = df_mass_crash[column] / df_mass_crash['POPULATION']

In [ ]:
# Drop unnessary columns
df_mass_crash = df_mass_crash.drop(['OBJECTID', 'CRASH_NUMB', 'NUMB_VEHC', 'NUMB_NONFATAL_INJR',
       'NUMB_FATAL_INJR', 'MILEMARKER', 'X', 'Y', 'LAT', 'LON', 'YEAR',
       'DISTRICT_NUM', 'SPEED_LIMIT', 'AADT', 'AADT_YEAR', 'PK_PCT_SUT',
       'AV_PCT_SUT', 'PK_PCT_CT', 'AV_PCT_CT', 'LT_SIDEWLK', 'RT_SIDEWLK',
       'SHLDR_LT_W', 'SURFACE_WD', 'SHLDR_RT_W', 'NUM_LANES', 'OPP_LANES',
       'MED_WIDTH', 'PEAK_LANE', 'SPEED_LIM', 'STATN_NUM', 'OP_DIR_SL',
       'SHLDR_UL_W', 'VEHC_UNIT_NUMB', 'DRIVER_AGE', 'TOTAL_OCCPT_IN_VEHC',
       'PERS_NUMB', 'AGE', 'state', 'county', 'county_subdivision'], axis=1)

**Perform Exploratory Data Analysis (EDA) to check for multicollinearity among the independent variables in the dataset**

In [ ]:
# Check for NaN missing values
df_mass_crash.isna().sum()

**Use Pandas Profiling**

In [ ]:
pp.ProfileReport(df_mass_crash)

In [ ]:
# Drop independent variables that are highly correlated
df_mass_filtered = df_mass_crash.drop(['BIKE_TO_WORK_EST', 'BIKE_TO_WORK_MARG',
       'WALK_TO_WORK_EST', 'WALK_TO_WORK_MARG', 'DRIVE_SOLO_TO_WORK_EST',
       'DRIVE_SOLO_TO_WORK_MARG', 'CARPOOL_TO_WORK_EST',
       'CARPOOL_TO_WORK_MARG', 'PUBTRANS_TO_WORK_EST', 'PUBTRANS_TO_WORK_MARG',
       'BNA_SCORE', 'BICYCLE_DEATH_BY_AUTO_COUNT',
       'BICYCLE_SUSP_SERIOUS_INJ_BY_AUTO_COUNT', 'BICYCLE_BY_AUTO_COUNT',
       'AUTOMOBILE_COUNT', 'PED_DEATH_BY_AUTO_COUNT',
       'PED_SUSP_SERIOUS_INJ_BY_AUTO_COUNT', 'PED_BY_AUTO_COUNT'], axis=1)

### 2. Fit the linear regression using [scikit-learn](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html)

**Separate data set in Y(independent) and X (dependent) variable**

In [ ]:
y2 = df_mass_filtered["BICYCLE_BY_AUTO_COUNT_PER_CAPITA"] # Y = df_mass_filtered.BICYCLE_BY_AUTO_COUNT_PER_CAPITA
X2 = df_mass_filtered.loc[:, df_mass_filtered.columns != "BICYCLE_BY_AUTO_COUNT_PER_CAPITA"] # I want all columns except the BICYCLE_BY_AUTO_COUNT_PER_CAPITA column which is the dependent variable

**Use the train_test_split function to split your data into training (80%) and testing set (20%)**

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X2, y2, test_size=0.2, random_state=5)

**Fit, run or estimate the regression model**

In [ ]:
model = LinearRegression() # Create an instance of the linear regression class
model.fit(X_train, y_train)

In [ ]:
model.coef_

In [ ]:
print(model.intercept_, model.coef_,model.score(X_test, y_test)) # Model score gives you the R-squared

**Use the independent variables in the testing set, to predict the dependent variables**

In [ ]:
y_pred = model.predict(X_test)

**Use the test set, check how well the model does in terms of error metrics**

In [ ]:
MAE = mean_absolute_error(y_test,y_pred)
MSE = mean_squared_error(y_test,y_pred)
MAPE =  mean_absolute_percentage_error(y_test,y_pred)
MSE, MAE, MAPE

### 3. Fit the linear regression using [Statsmodels](https://www.statsmodels.org/stable/index.html)

In [ ]:
X_train = sm.add_constant(X_train)
model2 = sm.OLS(y_train, X_train).fit()

In [ ]:
print(model2.summary())

**According to the results from Statsmodels, in the 80% training set, the estimated coefficients, including BICYCLE_DEATH_BY_AUTO_COUNT_PER_CAPITA, BICYCLE_SUSP_SERIOUS_INJ_BY_AUTO_COUNT_PER_CAPITA, PED_BY_AUTO_COUNT_PER_CAPITA, PED_DEATH_BY_AUTO_COUNT_PER_CAPITA, PED_SUSP_SERIOUS_INJ_BY_AUTO_COUNT_PER_CAPITA,  are statistically significant at the 5% level of significance.**

**In-sample prediction**

In [ ]:
ypred2 = model2.predict(X_train)
model2.params

### 4. Plot the residuals, display feature coefficients, and create a QQ plot

### Credit:

The following code is based on the work of my supervisor, Mitch Shiles. The original code can be found at this link: [Mitch Shiles' GitHub](https://github.com/rmshiles/Custom-Data-Tools/blob/main/testing%20for%20normality%20.ipynb).

In [ ]:
# Plot the residuals, display feature coefficients, and create a QQ plot
test_for_normality(y_test,y_pred)

**The warning regarding the kurtosis test indicates that it's typically valid for larger sample sizes (n >= 20). With a sample size of 9, it's recommended to use other statistical tests or methods to assess kurtosis accurately.**